# Communication carte micro:bit broker MQTT

L'objectif de cette activité est de faire communiquer un objet connecté avec un smartphone en utilisant le protocole MQTT. 

Le principe de la communication est le suivan. Un objet (del, capteur de température, ..) est branché sur la carte micro: bit . La carte micro:bit communique avec le PC.  Le PC se charge de communiquer avec un broker MQTT . Le smartphone communique avec le broker MQTT.

       port série/USB                python                  broker MQTT                        client MQTT
         microbit <---------------->   PC    <------------>  (Internet ou PC)   <-------------->  smartphone
 

Prérequis
- le client MQTT est installé sur les postes : commande pip install paho-mqtt 
- travail préparatoire sur le broker MQTT : [broker mosquitto](https://mosquitto.org "https://mosquitto.org/") : création de topcs, souscription, publication (cf activité 3 : le protocole MQTT)
- utilisation du smartphone pour envoyer/recevoir des données : les élèves doivent avoir installé un client MQTT.


## Activités
- activité 1 : communication ordinateur / broker MQTT
    - envoyer un message au broker depuis l'ordinateur 
    - récupérer ce message depuis un smartphone

- activité 2 : envoyer /recevoir des messages depuis la carte micro:bit
    - envoyer un message : la carte l'affiche avec ses LEDS
    - Presser sur "bouton A" ou "bouton B" sur la carte : le téléphone reçoit un message "bouton A" ou "bouton B"

- activité 3 : gestion de la luminosité
    - si la luminosité est en dessous d'un certain seuil, un message "faible luminosité" est envoyé sur le portable
    - depuis le portable, allumer une led branchée surla carte.


La carte est connectée au python du poste prof via un port série (émulé sur port USB). 



## Configuration du client MQTT
- ouvrir IOT MQTT Panel sur le smartphone
- paramétrer une nouvelle connexion : se connecter au broker mosquitto.org ( https://mosquitto.org )
- Port d'écoute du broker : 1883 .



Pour faire communiquer la carte micro:bit et le broker MQTT  (activités 2 et 3) 4 codes sont nécessaires, à executer **dans cet ordre** sur le poste  

1. le code à télécharger sur la carte microbit 
    - copier-coller le code python dans l'éditeur [python pour microbit](https://python.microbit.org/v/2.0 " https://python.microbit.org/v/2.0 ")
    - Dans l'éditeur python pour microbit, cliquer sur "Download"
    - Glisser-Déposer le fichier microbit_program.hex téléchargé sur la carte microbit.
2. le code pour trouver le port sur lequel est connecté la carte micro:bit (cliquer sur la cellule et "executer")
3. le code à lancer sur le pc pour communiquer 
4. le code à lancer pour clore la communication

##  Pour voir sur quel port  du PC est connecté la carte micro:bit

Selon les PC, votre carte sera connectée sur le port série 'COM3' ou le port série 'COM7' ou le port série 'COM123'. Bref, il faut voir quel est le port série sur lequel se trouve la microbit. En exécutant le code ci-dessous on affiche toutes les infos sur les ports série disponibles / ouverts.  
Sur mon PC, un seul en USB : donc c'était lui ('COM7') sur lequel était la microbit.

(cliquer sur la cellule puis "executer")

In [1]:
import serial.tools.list_ports as sr_tl

def scanner_ports_serie_pour_trouver_microbit():
    K = sr_tl.comports()
    for portInfo in K:
        print('----------------------')
        print(portInfo.device)
        print(portInfo.name)
        print(portInfo.description)
        print(portInfo.hwid)

scanner_ports_serie_pour_trouver_microbit()

----------------------
COM3
COM3
Périphérique série USB (COM3)
USB VID:PID=0D28:0204 SER=9901000050784E45000F10100000001F0000000097969901 LOCATION=1-1:x.1


## Pour clore la communication avec le port série

Il est nécessaire de clore la communication proprement, sinon le port série est "immobilisé" et on ne peut plus s'y reconnnecter. Il faut alors débrancher la carte micro:bit du PC et la rebrancher.

Il faut exécuter le programme suivant


In [46]:
import time
continuer_lecture = False   # on coupe la boucle puis la comm 
time.sleep(1)
COMM_SERIE.close()                       # sinon readline sur une comm coupée --> erreur
CLIENT_MQTT.unsubscribe(TOPIC_MQTT_2_MB)    # désinscription du topic
CLIENT_MQTT.loop_stop()         # clot la communication avec le broker

## Activité 1 : communication ordinateur / broker MQTT

1. Exécuter le programme ci dessous

In [10]:
#code python à exécuter sur l'ordinateur (cliquer sur la cellule, puis "executer")

from threading import Thread
import paho.mqtt.client as mqtt
import time
  
    

def demarrer_client_mqtt(client):
    client.connect_async(BROKER_IP, BROKER_PORT, 60)
    client.loop_start()


def test_envoi(chaine_car):
    CLIENT_MQTT.publish(TOPIC, payload = chaine_car)


BROKER_IP = "test.mosquitto.org" 
BROKER_PORT = 1883
TOPIC= 'SNTLFV'
CLIENT_MQTT = mqtt.Client()
demarrer_client_mqtt(CLIENT_MQTT)
time.sleep(1)
CLIENT_MQTT.subscribe(TOPIC)

(0, 1)

2. Se connecter au broker MQTT
 - créer un panel "Text Log"
 - trouver le nom du topic à créer d'après le programme ci-dessus

3. Executer le code suivant (cliquer sur la cellule, puis "executer"). Récupérer le message sur votre portable. Changer le topic dans le programme du dessus et le message. Récupérer le nouveau message sur le nouveau topic.

In [20]:
test_envoi("salut")

## Activité 2 : envoyer/recevoir des messages depuis/vers la carte micro:bit

### Code à télécharger sur la carte micro:bit

``` python
import microbit

microbit.uart.init(baudrate=115200, bits=8, parity=None, stop=1)

while microbit.running_time() < 3600000:
    
    donnees = microbit.uart.readline()
    if donnees != None:
        message_recu = str(donnees, 'utf-8')
        microbit.display.show(message_recu)
        
    if microbit.button_a.is_pressed():
        microbit.uart.write('bouton A')
        microbit.sleep(1000)
        
    if microbit.button_b.is_pressed():
        microbit.uart.write('bouton B')
        microbit.sleep(1000)
```

### Code à éxécuter sur le pc

Modifier le port COM en fonction du port COM détecté par le code situé plus haut.

Penser à clore la communication avec le code situé plus haut

Grâce au module paho.mqtt.client, on crée un client MQTT : il s'agit d'un "objet" permettant d'assurer la communication avec le serveur (ou broker) MQTT.

Ce "client" doit d'abord être connecté au broker grâce à la méthode demarrer_client().
Ensuite on dispose de quatre méthodes :
- `souscrire` permet de s'abonner à un topic.
- `on_message` est automatiquement appelée à la réception d'un message : affichage du topic et du message.
- `desabonner`  permet de cesser son abonnement à un topic donné.
- `publier`   permet de publier un message sur un topic qui est précisé.


In [ ]:
import serial
from threading import Thread
import paho.mqtt.client as mqtt
import time

    
def on_connect_mqtt(client, userdata, flags, rc):
    print("Connexion asynchrone effectuée avec le code résultant : " + str(rc))
    
def on_message_mqtt(client, userdata, msg):
    message = msg.payload.decode('utf-8')
    print(message)
    COMM_SERIE.write(message.encode("utf-8"))

def demarrer_client_mqtt(client):
    client.on_connect = on_connect_mqtt
    client.on_message = lambda c, u, m : on_message_mqtt(c, u, m)
    client.connect_async(BROKER_IP, BROKER_PORT, 60)
    client.loop_start()


def ecoute_microbit(comm):
    while continuer_lecture:
        data = comm.readline().decode("utf-8")
        if data:
            print(data)
            CLIENT_MQTT.publish(TOPIC_MB_2_MQTT, payload = data)


BROKER_IP = "test.mosquitto.org" 
BROKER_PORT = 1883
TOPIC_MB_2_MQTT= 'SNTLFV/microbit/depuis_microbit'
TOPIC_MQTT_2_MB= 'SNTLFV/microbit/vers_microbit'
CLIENT_MQTT = mqtt.Client()
demarrer_client_mqtt(CLIENT_MQTT)
time.sleep(1)
CLIENT_MQTT.subscribe(TOPIC_MQTT_2_MB)

COMM_SERIE = serial.Serial('COM3', baudrate = 115200, bytesize = serial.EIGHTBITS, 
                           parity = serial.PARITY_NONE, stopbits = serial.STOPBITS_ONE, 
                           timeout=.1, write_timeout = 0)

continuer_lecture = True
thread_ecoute_microbit = Thread(target = lambda : ecoute_microbit(COMM_SERIE))
thread_ecoute_microbit.start()

1. Executer le code précédent (cliquer sur la cellule, puis "executer") 
Lors de l'apui sur les boutons A ou B, la micro:bit envoie ses infos sur le port série. Python récupère ces infos et les diffuse au broker sur le topic "depuis_microbit". Depuis le client MQTT du smartphone, on écoute la carte micro:bit sur le topic "SNTLFV/depuis_microbit".

Sur le client MQTT du smartphone :
 - créer un panel de type "Text Log" . Trouver le nom du topic en fonction du programme ci-dessus
 - appuyer sur le bouton A ou B. Vérifier qu'on récupère le message sur le portable.


2. On envoie à la carte micro:bit sur le topic  "SNTLFV/vers_microbit" 

Envoyer un message depuis le client MQTT du smartphone :
  - créer un panel de type "Text Input". Trouver le nom du topic en fonction du programme ci-dessus.
  - Envoyer un message sur la carte depuis votre portable. Vérifier que le message s'affiche sur la carte micro:bit.

**Clore la communication**

## Activité 3 : gestion de la luminosité

- Carte micro:bit
    - capteur de lumière sur le pin0
    - del rouge  sur le pin1
    - del verte sur le pin2

- Pour faire allumer/eteindre leds :
    - publier sur le topic : SNTLFV/microbit/vers_microbit/leds/rouge ou SNTLFV/microbit/vers_microbit/leds/verte
    - le message "allume" pour allumer
    - le message "eteint" pour éteindre

- Pour recevoir valeur de la luminosité : s'abonner au topic : SNTLFV/depuis_microbit/luminosite




### Code à télécharger sur la carte microbit

```python

import microbit

microbit.uart.init(baudrate=115200, bits=8, parity=None, stop=1)

while microbit.running_time() < 36000000: #10 heures
    
    donnees = microbit.uart.readline()
    if donnees != None:
        message_recu = str(donnees, 'utf-8')
        if message_recu == 'led_rouge_allume':
            microbit.pin1.write_digital(1)
        elif message_recu == 'led_rouge_eteint':
            microbit.pin1.write_digital(0)  
        elif message_recu == 'led_verte_allume':
            microbit.pin2.write_digital(1)
        elif message_recu == 'led_verte_eteint':
            microbit.pin2.write_digital(0)
            
    lum = microbit.pin0.read_analog()
    lumi = str(lum)
    microbit.uart.write(lumi)
    microbit.sleep(500)

```

### Code à éxécuter sur le pc

Cliquer sur la cellule puis "éxécuter"

Modifier le port COM en fonction du port COM détecté par le code situé plus haut.

Penser à clore la communication avec le code situé plus haut

In [45]:
import serial
from threading import Thread
import paho.mqtt.client as mqtt
import time

    
def on_connect_mqtt(client, userdata, flags, rc):
    print("Connexion asynchrone effectuée avec le code résultant : " + str(rc))
    
def on_message_mqtt(client, userdata, msg):
    message = msg.payload.decode('utf-8')
    if msg.topic == TOPIC_MQTT_2_MB[:-2] + '/rouge' :
        message = 'led_rouge_' + message
        COMM_SERIE.write(message.encode("utf-8"))
    elif msg.topic == TOPIC_MQTT_2_MB[:-2] + '/verte' :
        message = 'led_verte_' + message
        COMM_SERIE.write(message.encode("utf-8"))   

def demarrer_client_mqtt(client):
    client.on_connect = on_connect_mqtt
    client.on_message = lambda c, u, m : on_message_mqtt(c, u, m)
    client.connect_async(BROKER_IP, BROKER_PORT, 60)
    client.loop_start()


def ecoute_microbit(comm):
    while continuer_lecture:
        data = comm.readline().decode("utf-8")
        if data:
            #print("message reçu de port série et envoyé sur broker/depuis_microbit :", data)
            CLIENT_MQTT.publish(TOPIC_MB_2_MQTT + '/luminosite', payload = data)


BROKER_IP = "test.mosquitto.org" 
BROKER_PORT = 1883
TOPIC_MB_2_MQTT= 'SNTLFV/microbit/depuis_microbit'
TOPIC_MQTT_2_MB= 'SNTLFV/microbit/vers_microbit/leds/#'
CLIENT_MQTT = mqtt.Client()
demarrer_client_mqtt(CLIENT_MQTT)
time.sleep(1)
CLIENT_MQTT.subscribe(TOPIC_MQTT_2_MB)

COMM_SERIE = serial.Serial('COM3', baudrate = 115200, bytesize = serial.EIGHTBITS, 
                           parity = serial.PARITY_NONE, stopbits = serial.STOPBITS_ONE, 
                           timeout=.1, write_timeout = 0)

continuer_lecture = True
thread_ecoute_microbit = Thread(target = lambda : ecoute_microbit(COMM_SERIE))
thread_ecoute_microbit.start()

Connexion asynchrone effectuée avec le code résultant : 0
